In [1]:
#%matplotlib widget

import os
from feature import feature
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from skimage import morphology,draw
import cv2

f_path = "./traindata/fu/"
nf_path = "./traindata/not_fu/"
f_name = [f_path + i for i in os.listdir(f_path)]
nf_name = [nf_path + i for i in os.listdir(nf_path)]
t_path = "./testdata/"
t_name = [t_path + "000{0}.png".format(i + 1) for i in range(9)]
t_name.append(t_path + "0010.png")

def readimage(t_name):
    image = cv2.imread(t_name,cv2.IMREAD_GRAYSCALE)
    ret,image=cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    image=255-image
    image=image/255

    #实施骨架算法
    skeleton =morphology.skeletonize(image)

    image=image*255
    skeleton=255-skeleton
    image=255-image
    skeleton[skeleton!=255]=0
    return skeleton

def features0(t_name):
    # img = cv2.imread(t_name,cv2.IMREAD_GRAYSCALE)
    # ret,img=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    img=readimage(t_name)

    sumleft=np.zeros((10))
    for i in range(10):
        for x in range(50):
            for y in range(500):
                if img[x+i*50][y]==0:
                    sumleft[i]+=y

    sumright=np.zeros((10))
    for i in range(10):
        for x in range(50):
            for y in range(500):
                if img[x+i*50][499-y]==0:
                    sumright[i]+=y

    sumup=np.zeros((10))
    for i in range(10):
        for y in range(50):
            for x in range(500):
                if img[x][y+i*50]==0:
                    sumup[i]+=x

    sumdown=np.zeros((10))
    for i in range(10):
        for y in range(50):
            for x in range(500):
                if img[499-x][y+i*50]==0:
                    sumdown[i]+=x

    feature2=np.concatenate((sumleft,sumright,sumup,sumdown),axis=0)
    sumleft3=np.zeros((10))
    for i in range(10):
        for x in range(50):
            flag=0
            begin=0
            for y in range(500):
                if img[x+i*50][y]==0 and flag==0:
                    flag+=1
                if img[x+i*50][y]==255 and flag==1:
                    flag+=1
                    begin=y
                if img[x+i*50][y]==0 and flag==2:
                    flag+=1
                    sumleft3[i]+=y-begin

    sumright3=np.zeros((10))
    for i in range(10):
        for x in range(50):
            flag=0
            begin=0
            for y in range(500):
                if img[x+i*50][499-y]==0  and flag==0:
                    flag+=1
                if img[x+i*50][499-y]==255  and flag==1:
                    flag+=1
                    begin=y
                if img[x+i*50][499-y]==0  and flag==2:
                    flag+=1
                    sumright3[i]+=y-begin

    sumup3=np.zeros((10))
    for i in range(10):
        for y in range(50):
            flag=0
            begin=0
            for x in range(500):
                if img[x][y+i*50]==0 and flag==0:
                    flag+=1
                if img[x][y+i*50]==255 and flag==1:
                    flag+=1
                    begin=x
                if img[x][y+i*50]==0 and flag==2:
                    flag+=1
                    sumup3[i]+=x-begin

    sumdown3=np.zeros((10))
    for i in range(10):
        for y in range(50):
            flag=0
            begin=0
            for x in range(500):
                if img[499-x][y+i*50]==0 and flag==0:
                    flag+=1
                if img[499-x][y+i*50]==255 and flag==1:
                    flag+=1
                    begin=x
                if img[499-x][y+i*50]==0 and flag==2:
                    flag+=1
                    sumdown3[i]+=x-begin

    feature3=np.concatenate((sumleft3,sumright3,sumup3,sumdown3),axis=0)
    feature23=np.concatenate((feature2/25,feature3),axis=0)
    return feature23

In [8]:
nf_name[0]

'./traindata/not_fu/notfu (97).png'

In [3]:
from feature import match
from sklearn import svm
from sklearn.svm import OneClassSVM

X_train=[]
y_train=[]
for i in range(len(nf_name)):
    X_train.append(features0(nf_name[i]))
    y_train.append([0])
for i in range(len(f_name)):
    X_train.append(features0(f_name[i]))
    y_train.append([1])






In [4]:
X_test=[]
for i in range(10):
    X_test.append(features0(t_name[i]))


In [9]:
X_test[0]

array([2129.56, 2256.68, 5851.76, 4219.68, 4731.36, 4834.56, 4558.72,
       4707.04, 7932.4 ,  989.04, 1103.96, 3332.12, 3569.36, 5181.48,
       5607.92, 4047.64, 4044.04, 4993.52, 7935.8 , 1066.84, 1979.24,
       3558.44, 1787.2 , 3765.72, 9808.88, 8991.08, 7414.24, 4531.04,
       1315.28, 1204.76,  655.48, 2210.  , 3621.96, 3280.16, 5959.52,
       8434.  , 9072.72, 4011.84,  760.56,  731.36,  337.  , 4659.  ,
       3943.  , 3520.  , 2838.  , 5909.  , 5353.  , 2620.  , 2635.  ,
        111.  ,  579.  , 5862.  , 1414.  , 2522.  , 1937.  , 7119.  ,
       5837.  , 3485.  , 1272.  ,   82.  , 2931.  , 6117.  , 1995.  ,
       5815.  , 6980.  , 3707.  ,  981.  , 5105.  , 5498.  ,  674.  ,
       1733.  , 4505.  , 3447.  , 3044.  ,  930.  ,  617.  , 1109.  ,
       1302.  , 5409.  ,  674.  ])

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
clf = svm.SVC(kernel='poly')
clf.fit(X_train1, y_train1)
y_pred = clf.predict(X_test1)
accuracy = accuracy_score(y_test1, y_pred)
accuracy

/home/zhaohui/anaconda3/envs/cjl/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9871794871794872

In [7]:
clf = svm.SVC(kernel='poly')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred

/home/zhaohui/anaconda3/envs/cjl/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])